In [78]:
#file names
file_1 = "train.csv"
file_2 = "test.csv"

In [79]:
import pandas as pd
import matplotlib as plt

f_1 = pd.read_csv(file_1)
f_2 = pd.read_csv(file_2)

#checking the missing values from our dataset 
missing_values_after = f_1.isnull().sum() 
print("the missing values after filling:")
print(missing_values_after)

#f_1.dropna(inplace = True) remove them
print(f_1.dtypes)
#filling the missing values that are missing
f_1["LotFrontage"].fillna(f_1["LotFrontage"].mean(), inplace = True)
missing_values_before = f_1.isnull().sum()
print("the missing values before filling: ")
print(missing_values_before)

the missing values after filling:
Id                 0
MSSubClass         0
MSZoning           0
LotFrontage      259
LotArea            0
                ... 
MoSold             0
YrSold             0
SaleType           0
SaleCondition      0
SalePrice          0
Length: 81, dtype: int64
Id                 int64
MSSubClass         int64
MSZoning          object
LotFrontage      float64
LotArea            int64
                  ...   
MoSold             int64
YrSold             int64
SaleType          object
SaleCondition     object
SalePrice          int64
Length: 81, dtype: object
the missing values before filling: 
Id               0
MSSubClass       0
MSZoning         0
LotFrontage      0
LotArea          0
                ..
MoSold           0
YrSold           0
SaleType         0
SaleCondition    0
SalePrice        0
Length: 81, dtype: int64


C:\Users\walid\AppData\Local\Temp\ipykernel_6128\3964314522.py:15: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  f_1["LotFrontage"].fillna(f_1["LotFrontage"].mean(), inplace = True)


In [80]:
#hundeling -seperating- types of data we have
numerical_columns = f_1.select_dtypes(include=['int64','float64']).columns.tolist()
categorical_columns = f_1.select_dtypes(include= ['object']).columns.tolist()
#binary_columns = [col for col in f_1 if f_1[col].nunique() == 2]
#categorical_columns_cleaned = [col for col  in categorical_columns if col not in binary_columns]
print("numerical columns : ", numerical_columns)
print("categorical columns :", categorical_columns)
#print("binary columns : ", binary_columns)
numerical_set = set(numerical_columns)
categorical_set = set(categorical_columns)
#binary_set = set(binary_columns)
numerical_categorical_intersection = numerical_set.intersection(categorical_set)
#binary_categorical_intersection = binary_set.intersection(categorical_set)
#binary_numerical_intersection = binary_set.intersection(numerical_set)

print(numerical_categorical_intersection  )
alpha =  'PoolQC' in categorical_columns
print(alpha)
alpha_1 = f_1['SalePrice'].min()
print(alpha_1)

numerical columns :  ['Id', 'MSSubClass', 'LotFrontage', 'LotArea', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd', 'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'TotRmsAbvGrd', 'Fireplaces', 'GarageYrBlt', 'GarageCars', 'GarageArea', 'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea', 'MiscVal', 'MoSold', 'YrSold', 'SalePrice']
categorical columns : ['MSZoning', 'Street', 'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'Heating', 'HeatingQC', 'CentralAir', 'Electrical', 'KitchenQual', 'Functional', 'Firepl

In [81]:
#feature engineering step
f_1_encoded = pd.get_dummies(f_1, columns = categorical_columns, drop_first=True)

#scaling numerical values
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
f_1_encoded[numerical_columns] = scaler.fit_transform(f_1_encoded[numerical_columns])
listremain = [col for col in f_1_encoded if f_1_encoded[col].dtypes == object]


In [82]:
#sale_price is our target
X_train = f_1_encoded.drop('SalePrice', axis=1)
y_train = f_1_encoded['SalePrice']

In [83]:
#trainning phase
from sklearn.linear_model import LinearRegression

model = LinearRegression()

model.fit(X_train, y_train)

ValueError: Input X contains NaN.
LinearRegression does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values